In [2]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [3]:
import json

with open("task_pair_metrics_3.jsonl") as f:
    lines = f.readlines()
    results = [json.loads(line) for line in lines]

In [4]:
first_task = "antonyms"

task_results = [result for result in results if result["task_pair"][0] == first_task]

len(task_results)

44

In [5]:
from sprint.task_vector_utils import load_tasks, ICLRunner

# Load tasks
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [11]:
task_names = list(tasks.keys())

In [7]:
result_dict = {tuple(x["task_pair"]):x for x in results}

In [8]:
faithfulness_task = result_dict[(first_task, first_task)]["faithfullness"]
n_nodes_task = result_dict[(first_task, first_task)]["n_nodes_counts"]



In [9]:
for second_task in task_names:
    try:
        orig_metric = result_dict[(second_task, second_task)]["orig_metric"]
        zero_metric = result_dict[(second_task, second_task)]["zero_metric"]

        print(
            f"{second_task}: orig_metric: {orig_metric}, zero_metric: {zero_metric}"
        )
    except KeyError:
        print(f"No results for {second_task}")

location_continent: orig_metric: -30.625, zero_metric: -80.0
football_player_position: orig_metric: -66.0, zero_metric: -78.5
location_religion: orig_metric: -19.0, zero_metric: -80.0
location_language: orig_metric: -28.125, zero_metric: -93.0
person_profession: orig_metric: -75.5, zero_metric: -79.0
location_country: orig_metric: -62.0, zero_metric: -104.5
country_capital: orig_metric: -27.875, zero_metric: -116.5
person_language: orig_metric: -27.0, zero_metric: -69.5
singular_plural: orig_metric: -18.0, zero_metric: -90.5
present_simple_past_simple: orig_metric: -15.25, zero_metric: -90.5
antonyms: orig_metric: -18.375, zero_metric: -76.0
plural_singular: orig_metric: -17.75, zero_metric: -87.5
present_simple_past_perfect: orig_metric: -30.25, zero_metric: -88.0
present_simple_gerund: orig_metric: -13.3125, zero_metric: -83.5
en_it: orig_metric: -56.0, zero_metric: -176.0
it_en: orig_metric: -51.25, zero_metric: -75.5
en_fr: orig_metric: -77.0, zero_metric: -183.0
en_es: orig_metric

In [13]:
import plotly.express as px

first_task = "antonyms"

faithfulness_task = result_dict[(first_task, first_task)]["faithfullness"]
n_nodes_task = result_dict[(first_task, first_task)]["n_nodes_counts"]


fig = px.line(x=[max(n_nodes_task) - x for x in n_nodes_task], y=faithfulness_task)

for second_task in task_names:
    if second_task == "person_profession":
        continue
    faithfulness_second = result_dict[(first_task, second_task)]["faithfullness"]
    n_nodes_second = result_dict[(first_task, second_task)]["n_nodes_counts"]
    fig.add_scatter(x=[max(n_nodes_second) - x for x in n_nodes_second], y=faithfulness_second, name=second_task)

# fig.add_scatter(x=[max(n_nodes_second) - x for x in n_nodes_second], y=faithfulness_second)

fig.update_xaxes(title="Number of nodes")
fig.update_yaxes(title="Faithfullness")

fig


In [24]:
import numpy as np 

task_names = [x for x in task_names if x not in ["person_profession", "football_player_position"]]
heatmap_data = np.zeros((len(task_names), len(task_names)))


for i, first_task in enumerate(task_names):
    faith = result_dict[(first_task, first_task)]["faithfullness"]
    idx = [i for i, x in enumerate(faith) if x < 0.5][0]
    for j, second_task in enumerate(task_names):
        faith = result_dict[(first_task, second_task)]["faithfullness"]
        heatmap_data[i, j] = faith[idx]

fig = px.imshow(heatmap_data, x=task_names, y=task_names)

fig